### Find the maximum impacts for individual, local, synergistic and global nudges

In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import numpy as np
from scipy import stats

import probability_distributions
import maximum_nudges
import evolutionary_algorithms as ea
import maximum_nudges_evolutionary as ev_max_nudges

#### First generate a generic input distribution

In [ ]:
#distribution parameters
input_variables = 4
number_of_states = 5
nudge_size = 0.01

#generate both input and conditional output with Dirichlet weights
distribution_shape = [number_of_states]*input_variables
total_number_of_states = reduce(lambda x,y: x*y, distribution_shape)
input_dist = np.random.dirichlet([1]*total_number_of_states)
input_dist = np.reshape(input_dist, distribution_shape)
cond_shape = [number_of_states]*(input_variables+1)
cond_output = [
    probability_distributions.compute_joint_uniform_random((number_of_states,))
    for i in range(number_of_states**(input_variables))
]
cond_output = np.array(cond_output)
cond_output = np.reshape(cond_output, cond_shape)


#### Generate a generic input distribution with limited entropy

In [ ]:
shape = [5]*6
dist = ea.produce_distribution_with_entropy(shape, 0.7)


In [ ]:
print(stats.entropy(dist, base=2))
#print(dist[1000:2000])

#### Define the parameters for the evolutionary algorithm to optimize the impact of a local nudge

In [ ]:
#local nudge optimization
number_of_generations = 200 
population_size = 10
number_of_children = 20 
generational = True 
mutation_size = nudge_size/4
parent_selection_mode = "rank_exponential"
#parent_selection_mode = None
mutation_size_weights = 0.03
start_mutation_size = nudge_size/10
change_mutation_size = start_mutation_size/10
nudged_vars_to_states = {
    nudged_var:number_of_states for nudged_var in range(input_variables)
}


In [ ]:
ev_max_nudges.TEST = False

#create the initial population
local_nudges = []
for _ in range(population_size):
    new_local_nudge = ev_max_nudges.LocalNudge.create_local_nudge(
        nudged_vars_to_states, nudge_size, mutation_size_weights,
        start_mutation_size, change_mutation_size, timestamp=0
    )
    local_nudges.append(new_local_nudge)

for local_nudge in local_nudges:
    local_nudge.evaluate(input_dist, cond_output)
print("initial impact local nudge {}".format(
    ea.sort_individuals(local_nudges)[0].score
))

#start the optimization process
find_max_local_nudge = ev_max_nudges.FindMaximumLocalNudge(
    input_dist, cond_output, nudge_size, 
    generational, number_of_children, parent_selection_mode
)
max_local_nudge_individual = find_max_local_nudge.get_max_nudge(
    local_nudges, number_of_generations
)
print("the found max impact for a local nudge {}".format(
    max_local_nudge_individual.score
))


#### Find the maximum individual impact evolutionary

#### Find the maximum impact of an individual nudge exactly

In [ ]:
#still write some code to do the optimization for every variable
new_input_dist = np.copy(input_dist)
max_impacts = []
for i in range(len(new_input_dist.shape)):
    new_input_dist = np.swapaxes(new_input_dist, i, 
                                 len(new_input_dist.shape)-1)
    max_impact = maximum_nudges.find_maximum_local_nudge(
        new_input_dist, cond_output, nudge_size/2
    )
    max_impacts.append(max_impact)
    new_input_dist = np.swapaxes(new_input_dist, i, 
                                 len(new_input_dist.shape)-1)
    
print("the actual maximum individual nudge {}".format(max(max_impacts)))